In [1]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline,BitsAndBytesConfig, LlamaForCausalLM, LlamaTokenizer, GenerationConfig,LlamaConfig,LlamaModel,AutoModelForCausalLM
import transformers
import torch
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.document_loaders import PDFPlumberLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
import os
import streamlit as st
from llama_index import Document, LangchainEmbedding, SimpleDirectoryReader, ListIndex,VectorStoreIndex, StorageContext, GPTVectorStoreIndex, LLMPredictor, ServiceContext, load_index_from_storage
#from llama_index.vector_stores import DeepLakeVectorStore
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import base64
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
import torch.cuda as cuda

from langchain.agents.agent_toolkits import (VectorStoreInfo, create_vectorstore_agent, VectorStoreToolkit)

/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.12) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
model_id = 'meta-llama/Llama-2-70b-chat-hf'

In [3]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb configuration for quantization of the model in 4bits

In [5]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
model = LlamaForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit 

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/ubuntu/LLM-as-a-Service/falcon_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit 

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 8192, padding_idx=0)
    (layers): ModuleList(
      (0-79): 80 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=8192, out_features=28672, bias=False)
          (up_proj): Linear4bit(in_features=8192, out_features=28672, bias=False)
          (down_proj): Linear4bit(in_features=28672, out_features=8192, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
configuration = LlamaConfig()
model = LlamaModel(configuration)
configuration = model.config
configuration

In [8]:

generation_config = GenerationConfig.from_pretrained(model_id)
generation_config   


GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.9,
  "top_p": 0.6,
  "transformers_version": "4.32.0.dev0"
}

In [9]:


model = model.eval()
     


In [34]:
prompt = "What is the difference in architacture between the gpt model and the llama large language model?"

In [35]:
encoding = tokenizer(prompt, return_tensors="pt").to(model.device)

In [36]:
%%time
with torch.no_grad():
    outputs = model.generate(
        **encoding,
        max_new_tokens=256,
        temperature=0,
        generation_config=generation_config,
    )

CPU times: user 2min, sys: 1min 7s, total: 3min 8s
Wall time: 3min 8s


In [37]:
outputs

tensor([[    1,  1724,   338,   278,  4328,   297,  3190,   277,   627,   545,
          1546,   278,   330,   415,  1904,   322,   278, 11148,  3304,  2919,
          4086,  1904, 29973,    13,    13, 22550, 29901,   450, 11258,   310,
           278,   402,  7982,  1904,   322,   278,   365,  5661,  1529,  2919,
          4086,  1904,   526,  2788,   297,  1784,  5837, 29892,   408,   896,
           526,  1716,  2729,   373,  4327,   261, 29899,  6707,  6956,  1973,
         29889,  2398, 29892,   727,   526,   777,  1820, 12651,  1546,   278,
          1023,  4733, 29889,    13,    13, 29896, 29889,  8125, 21179, 29901,
           450,  1556,  6924,  4328,   338,   278,  2159,   310,   278,  4733,
         29889,   402,  7982,   338,   263,  1568,  7968,  1904,  1135,   365,
          5661,  1529, 29892,   411, 14235, 29871, 29955, 29900,  7284,  4128,
          9401,   304,   365,  5661,  1529, 29915, 29879, 29871, 29896, 29889,
         29945, 24464,  4128, 29889,    13, 29906, 2

In [38]:
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

What is the difference in architacture between the gpt model and the llama large language model?

Answer: The architecture of the GPT model and the LLaMA large language model are similar in many ways, as they are both based on transformer-based architectures. However, there are some key differences between the two models.

1. Model Size: The most obvious difference is the size of the models. GPT is a much smaller model than LLaMA, with approximately 70 million parameters compared to LLaMA's 1.5 billion parameters.
2. Training Objective: GPT is trained with a language modeling objective, where the goal is to predict the next word in a sequence of text given the context of the previous words. LLaMA, on the other hand, is trained with a combination of a language modeling objective and a next sentence prediction objective. This means that LLaMA is trained to predict not only the next word in a sequence but also the next sentence in a sequence of text.
3. Training Data: GPT is trained on a 

In [14]:
answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
output_text = tokenizer.decode(answer_tokens[0], skip_special_tokens=True).strip()
print(output_text)

Open source LLMs have several advantages over ChatGPT, including:

Pros of Open Source LLMs:

1. Customizability: Open source LLMs can be customized and fine-tuned to suit specific use cases, allowing for greater flexibility and adaptability.
2. Cost-effectiveness: Open source LLMs are free and do not require expensive subscriptions or licensing fees, making them a cost-effective option for individuals and organizations.
3. Community-driven development: Open source LLMs are often developed and maintained by a community of developers, researchers, and users, which can lead to faster bug fixes, feature updates, and improvements.
4. Transparency: Open source LLMs provide a transparent view of the model's architecture, parameters, and training data, allowing for greater understanding and trust in the model's decision-making process.
5. Privacy: Open source LLMs can be trained on private data, ensuring that sensitive information remains confidential and is not shared with third-party servic

## Using Pipeline

In [12]:
prompt = "what are the danger of using AI?"
template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
USER: {prompt}
ASSISTANT:
'''

In [13]:
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15    
    
)

In [14]:
print(llama_pipeline(template)[0]['generated_text'])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
USER: what are the danger of using AI?
ASSISTANT:
There are several potential dangers associated with the use of artificial intelligence (AI). Some of these include:

1. Bias and discrimination: AI systems can perpetuate existing biases if they are trained on biased data or designed with a particular worldview. This can lead to unfair outcomes and discrimination against certain groups of people.
2. Job displacement: The increasing use of AI could potentially displace human worker

## Using HuggingFace Pipeline and LangChain

In [42]:

llm = HuggingFacePipeline(pipeline=llama_pipeline)

In [ ]:
prompt = "Who won the FIFA World Cup in the year 1994? "

template = '''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
USER: {prompt}
ASSISTANT:
'''

prompt_template = PromptTemplate(template=template, input_variables=["prompt"])